# Лабораторная работа 3. Генетические алгоритмы.

# Выполнили: Баринов Даниил, Бугров Лев, Голованов Денис

In [1]:
import time
import os
import random
from tabulate import tabulate

## Задача о рюкзаке

In [2]:
class KnapsackGeneticSolver():
    def __init__(self, w, p, W, population_size, num_groups, debug = False):
        self.w = w
        self.p = p
        self.W = W
        self.num_groups = num_groups
        self.size = len(w)
        self.population_size = population_size
        self.items = [i for i in range(self.size)]
        self.fitness_values = []
        self.population = []
        self.best = 0
        self.best_solution = []
        self.debug = debug
    
    
    def calc_fitness(self, chromosome):
        fitness_value = 0
        for i in range(self.size):
            fitness_value += chromosome[i] * self.p[i]
        if fitness_value >= self.best:
            self.best = fitness_value
            self.best_solution = chromosome.copy()
        return fitness_value
    
    
    def calc_weight(self, chromosome):
        sum_weight = 0
        for i in range(self.size):
            sum_weight += chromosome[i] * self.w[i]
        return sum_weight
    
    
    def create_chromosome(self):
        chromosome = []
        for i in range(self.size):
                chromosome.append(random.randint(0,1))
        chromosome = self.validate_chromosome(chromosome)
        return chromosome
    
    
    def validate_chromosome(self, chromosome):
        while (self.calc_weight(chromosome) > self.W):
            idx = random.randint(0,self.size-1)
            if chromosome[idx] == 1:
                chromosome[idx] = 0
        return chromosome
    
    
    def create_first_population(self):
        first_population = []
        for i in range(self.population_size):
            chromosome = self.create_chromosome()
            first_population.append(chromosome)
            self.fitness_values.append(self.calc_fitness(chromosome))
        self.population = first_population
        if (self.debug):
            print('First population:', first_population)
    
    
    def selection(self):
        population_fitness = [(i, self.fitness_values[i]) for i in range(self.population_size)]
        population_fitness.sort(key = lambda x: x[1], reverse = True)
        distribution = [0]*self.num_groups
        sum_fitness = sum(self.fitness_values)
        for i in range(self.num_groups):
            for j in range(self.population_size // self.num_groups):
                distribution[i] += population_fitness[i*(self.population_size // self.num_groups) + j][1]
            distribution[i] /= sum_fitness
        result = random.choices([i for i in range(self.num_groups)], distribution, k = 2)
        result = [result[0] + random.randint(0, self.population_size // self.num_groups), result[1] + random.randint(0, self.population_size // self.num_groups)]
        return result
    
    
    def crossover(self, first, second):
        point = random.randint(0, self.size - 1)
        for i in range(point, self.size):
            first[i], second[i] = second[i], first[i]
            
        return [self.validate_chromosome(first), self.validate_chromosome(second)]
    
    
    def mutation(self, chromosome):
        idx = random.randint(0, self.size - 1)
        if chromosome[idx] == 1:
            chromosome[idx] = 0
        else:
            chromosome[idx] = 1
        return self.validate_chromosome(chromosome)
            
        
    def execute(self, num_popualtions):
        self.create_first_population()
        for k in range(num_popualtions):
            i,j = self.selection()
            first_child, second_child = solver.crossover(self.population[i], self.population[j])
            
            if random.randint(0, 100) < 5:
                first_child = solver.mutation(first_child)
            if random.randint(0, 100) < 5:
                second_child = solver.mutation(second_child)

            self.fitness_values[i] = self.calc_fitness(first_child)
            self.fitness_values[j] = self.calc_fitness(second_child)
            
            self.population[i] = first_child
            self.population[j] = second_child
            
            if (self.debug):
                print('New population:', self.population)
                print('Fitness:', self.fitness_values)

In [3]:
def create_Knapsack_data(path, number):
    weights = []
    values = []
    answers = []
    W = 0
    for filename in sorted(os.listdir(path)):
        inp = open(path + '/' + filename).read()
        if str(number) in filename:
            if "_c" in filename:
                W = int(inp)
            elif "_p" in filename:
                values = list(map(int, inp.split()))
            elif "_s" in filename:
                answers = list(map(int, inp.split()))
            elif "_w" in filename:
                weights = list(map(int, inp.split()))
    return weights, values, answers, W

## Results

In [4]:
#Knapsack01 test

for i in range(1,7):
    weights, values, answers , W = create_Knapsack_data('benchmarks',i)
    solver = KnapsackGeneticSolver(weights,values, W, 8, 2)
    solver.execute(1000)
    correct_value = 0
    for i in range(len(answers)):
        correct_value += answers[i]*values[i]
    print(f'Correct value: {correct_value}')
        
    print(solver.best, solver.best_solution, answers)
    assert abs(solver.best - correct_value) <= correct_value / 2
    print('Success')

Correct value: 309
284 [1, 1, 0, 1, 0, 0, 1, 0, 0, 0] [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Success
Correct value: 51
51 [0, 1, 1, 1, 0] [0, 1, 1, 1, 0]
Success
Correct value: 150
150 [1, 1, 0, 0, 1, 0] [1, 1, 0, 0, 1, 0]
Success
Correct value: 107
107 [1, 0, 0, 1, 0, 0, 0] [1, 0, 0, 1, 0, 0, 0]
Success
Correct value: 900
900 [1, 0, 1, 1, 1, 0, 1, 1] [1, 0, 1, 1, 1, 0, 1, 1]
Success
Correct value: 1735
1735 [0, 1, 0, 1, 0, 0, 1] [0, 1, 0, 1, 0, 0, 1]
Success


In [5]:
results = {'benchmark_1':[], 'benchmark_2':[],'benchmark_3':[], 'benchmark_4':[],'benchmark_5':[], 'benchmark_6':[],'benchmark_7':[]}

for l in range(1, 8):
    weights, values, answers , W = create_Knapsack_data('benchmarks',l)

    correct_value = 0
    for i in range(len(answers)):
        correct_value += answers[i]*values[i]
    print(f'Correct value: {correct_value}')

    best_result = 0
    diff = correct_value
    
    start_time = time.time()
    for i in range(100):
        solver = KnapsackGeneticSolver(weights,values, W, 8, 2)
        solver.execute(1000)
        if abs(solver.best - correct_value) <= diff:
            diff = abs(solver.best - correct_value)
            best_result = solver.best
    end_time = time.time()

    results[f'benchmark_{l}'].append((round((end_time - start_time)/100, 5), best_result))

Correct value: 309
Correct value: 51
Correct value: 150
Correct value: 107
Correct value: 900
Correct value: 1735
Correct value: 1458


In [6]:
def display_table_of_results(results):
    table_time = [[' ', 'Knapsack Genetic Algorithm, 1000 populations']]
    
    for benchmark in results:
        table_time.append([benchmark] + list(map(lambda x: f'{x[0]} / {x[1]}', results[benchmark])))
    
    print('Measurments of running time of Knapsack genetic algorithm and results (time, s / answer)')
    print(tabulate(table_time, tablefmt='fancy_grid'))

In [7]:
display_table_of_results(results)

Measurments of running time of Knapsack genetic algorithm and results (time, s / answer)
╒═════════════╤══════════════════════════════════════════════╕
│             │ Knapsack Genetic Algorithm, 1000 populations │
├─────────────┼──────────────────────────────────────────────┤
│ benchmark_1 │ 0.02152 / 309                                │
├─────────────┼──────────────────────────────────────────────┤
│ benchmark_2 │ 0.01842 / 51                                 │
├─────────────┼──────────────────────────────────────────────┤
│ benchmark_3 │ 0.01829 / 150                                │
├─────────────┼──────────────────────────────────────────────┤
│ benchmark_4 │ 0.0217 / 107                                 │
├─────────────┼──────────────────────────────────────────────┤
│ benchmark_5 │ 0.02035 / 900                                │
├─────────────┼──────────────────────────────────────────────┤
│ benchmark_6 │ 0.02551 / 1735                               │
├─────────────┼──────────────

## Travelling salesman problem

In [8]:
def euc2d(x1,x2,y1,y2):
    return round(((x1-y1)**2 + (x2-y2)**2)**0.5,3)


class TSPGraph():
    def __init__(self, dist_matrix = None, coord_list = None):
        self.dist_matrix = dist_matrix
        self.coord_list = coord_list
        
        if not self.dist_matrix:
            self.create_dist_matrix()
    
    
    def create_dist_matrix(self):
        self.dist_matrix = []
        n = len(self.coord_list)
        for i in range(n):
            self.dist_matrix.append([0]*n)
        
        for i in range(n):
            for j in range(n):
                if i != j:
                    self.dist_matrix[i][j] = euc2d(self.coord_list[i][1],self.coord_list[i][2],self.coord_list[j][1],self.coord_list[j][2])

In [9]:
class TSPSolver():
    def __init__(self, dist_matrix, population_size, debug=False):
        self.dist = dist_matrix
        self.size = len(self.dist)
        self.population_size = population_size
        self.nodes = [i for i in range(self.size)]
        self.fitness_values = []
        self.population = []
        self.best = 0
        self.best_solution = []
        self.debug = debug
    
    def calc_fitness(self, chromosome):
        #print(chromosome)
        fitness_value = 0
        for i in range(self.size - 1):
            fitness_value += self.dist[i][i+1]
            
        fitness_value = 1 / (fitness_value + self.dist[chromosome[0]][chromosome[-1]])
        
        if fitness_value >= self.best:
            self.best = fitness_value
            self.best_solution = chromosome.copy()
            
        return fitness_value
    
    
    def create_chromosome(self):
        chromosome = self.nodes.copy()
        random.shuffle(chromosome)
        while chromosome in self.population:
            random.shuffle(chromosome)
        return chromosome
    
    
    def create_first_population(self):
        for i in range(self.population_size):
            chromosome = self.create_chromosome()
            self.population.append(chromosome)
            self.fitness_values.append(self.calc_fitness(chromosome))
    
    
    def roulette_wheel_selection(self):
        sum_fitness = sum(self.fitness_values)
        distribution = list(map(lambda x: x / sum_fitness,self.fitness_values))
        #print(distribution)
        result = random.choices([i for i in range(self.population_size)], distribution, k = 2)
        return result
    

    def crossover(self, first_parent, second_parent):
        first_child = [-1] * self.size
        second_child = [-1] * self.size
        
        cross_point1 = random.randint(0, self.size - 2)
        cross_point2 = random.randint(cross_point1 + 1, self.size - 1)
        
        first_child[cross_point1:cross_point2] = first_parent[cross_point1:cross_point2]
        second_child[cross_point1:cross_point2] = second_parent[cross_point1:cross_point2]
        
        first_order = []
        second_order = []
        
        for i in range(self.size):
            first_order.append(second_parent[(cross_point2 + i) % self.size])
            second_order.append(first_parent[(cross_point2 + i) % self.size])
            
        i = 0
        while i < len(first_order):
            if first_order[i] in first_child:
                first_order.pop(i)
            else:
                i += 1
        
        i = 0
        while i < len(second_order):
            if second_order[i] in second_child:
                second_order.pop(i)
            else:
                i += 1
        
        for i in range(len(first_order)):
            first_child[(cross_point2 + i) % self.size] = first_order[i]
        
        for i in range(len(second_order)):
            second_child[(cross_point2 + i) % self.size] = second_order[i]

        return first_child, second_child
        
        
    def mutation(self, parent, gap):
        child = parent.copy()
        j = random.randint(0, self.size - gap - 1)
        child[j:j + gap] = list(reversed(child[j:j + gap]))
        return child
    
    
    def execute(self, num_popualtions):
        self.create_first_population()
        for k in range(num_popualtions):
            i,j = self.roulette_wheel_selection()
            first_child, second_child = self.crossover(self.population[i], self.population[j])

            if random.randint(0, 100) == 1:
                first_child = self.mutation(first_child,5)
            if random.randint(0, 100) == 1:
                second_child = self.mutation(second_child,5)

            self.fitness_values[i] = self.calc_fitness(first_child)
            self.fitness_values[j] = self.calc_fitness(second_child)

            self.population[i] = first_child
            self.population[j] = second_child
            if self.debug:
                print('New population:', self.population)
                print('Fitness:', self.fitness_values)

In [10]:
def create_TSP_data(path, number):
    benchmarks = os.listdir(path)
    f = open(path + '/' + benchmarks[number])
    text = f.read()
    f.close()
    if 'NODE_COORD_SECTION' in text:
        coord_string = text[text.index('NODE_COORD_SECTION'):text.index('EOF'):]
        coord_arr = list(map(lambda s: s.replace('  ', ' ').strip().split(' '), coord_string.split('\n')[1:-1]))
        if '.' in coord_arr[0][1]:
            isInt = False
        else:
            isInt = True
        for i in range(len(coord_arr)):
            if '' in coord_arr[i]:
                coord_arr[i].remove('')
            if isInt:
                coord_arr[i] = list(map(int, coord_arr[i]))
            else:
                coord_arr[i] = list(map(float, coord_arr[i]))
        return coord_arr
    else:
        if 'DISPLAY_DATA_SECTION' in text:
            weight_matrix_string = text[text.index('EDGE_WEIGHT_SECTION'):text.index('DISPLAY_DATA_SECTION')]
        else:
            weight_matrix_string = text[text.index('EDGE_WEIGHT_SECTION'):text.index('EOF')]
        weight_matrix = list(map(lambda s:  s.replace('  ', ' ').strip().split(' '), weight_matrix_string.split('\n')[1:-1]))
        for i in range(len(weight_matrix)):
            if '' in weight_matrix[i]:
                weight_matrix[i].remove('')
            weight_matrix[i] = list(map(int, weight_matrix[i]))
        return weight_matrix

## Results

In [11]:
benchmarks = os.listdir('lab3_data')
results = {'a280.tsp': [], 'att48.tsp': [], 'bays29.tsp': [], 'ch150.tsp': [], 'fl417.tsp': []}

In [12]:
data = create_TSP_data('lab3_data', 0)
g = TSPGraph(coord_list=data)

start_time = time.time()
for i in range(10):
    s = TSPSolver(g.dist_matrix, 8)
    s.execute(1000)
end_time = time.time()

print(1 / s.best, s.best_solution)
results['a280.tsp'].append((round((end_time - start_time)/10, 5), round(1 / s.best, 3)))

2808.731999999999 [46, 242, 43, 68, 87, 190, 78, 204, 7, 147, 203, 100, 148, 107, 79, 264, 20, 215, 219, 81, 16, 44, 63, 123, 89, 108, 52, 111, 212, 142, 146, 29, 159, 223, 61, 33, 216, 267, 197, 217, 247, 179, 84, 42, 71, 59, 37, 95, 209, 125, 275, 236, 115, 192, 222, 180, 181, 188, 256, 276, 6, 48, 98, 55, 128, 65, 141, 182, 208, 244, 261, 220, 50, 72, 279, 228, 243, 213, 167, 24, 26, 90, 218, 269, 51, 114, 202, 5, 186, 4, 149, 102, 268, 105, 32, 150, 199, 214, 227, 62, 246, 122, 238, 1, 13, 250, 82, 92, 54, 36, 258, 172, 252, 2, 57, 229, 155, 130, 76, 157, 41, 235, 196, 224, 38, 163, 138, 14, 194, 17, 131, 119, 94, 144, 185, 23, 25, 135, 27, 277, 278, 266, 126, 230, 124, 11, 225, 187, 173, 140, 129, 113, 120, 117, 255, 8, 253, 118, 152, 272, 66, 112, 21, 271, 171, 96, 162, 176, 15, 145, 184, 39, 109, 116, 189, 35, 10, 234, 143, 263, 134, 239, 85, 77, 175, 34, 245, 0, 259, 164, 103, 265, 60, 166, 154, 233, 56, 207, 231, 200, 80, 226, 88, 110, 31, 241, 191, 183, 151, 86, 273, 47, 270,

In [13]:
data = create_TSP_data('lab3_data', 1)
g = TSPGraph(coord_list=data)

start_time = time.time()
for i in range(10):
    s = TSPSolver(g.dist_matrix, 8)
    s.execute(1000)
end_time = time.time()

print(1 / s.best, s.best_solution)
results['att48.tsp'].append((round((end_time - start_time)/10, 5), round(1 / s.best, 3)))

153918.63700000002 [18, 30, 41, 33, 20, 40, 5, 34, 21, 47, 44, 6, 27, 39, 4, 29, 9, 22, 12, 11, 43, 13, 46, 14, 8, 2, 3, 35, 17, 37, 45, 1, 32, 42, 0, 19, 24, 15, 10, 26, 7, 28, 38, 16, 23, 31, 25, 36]


In [14]:
data = create_TSP_data('lab3_data', 2)
g = TSPGraph(dist_matrix=data)

start_time = time.time()
for i in range(10):
    s = TSPSolver(g.dist_matrix, 8)
    s.execute(1000)
end_time = time.time()

print(1 / s.best, s.best_solution)
results['bays29.tsp'].append((round((end_time - start_time)/10, 5), round(1 / s.best, 3)))

5620.0 [17, 7, 20, 0, 22, 3, 8, 24, 27, 12, 4, 11, 6, 21, 25, 1, 28, 15, 23, 2, 26, 19, 14, 10, 16, 5, 9, 18, 13]


In [15]:
data = create_TSP_data('lab3_data', 3)
g = TSPGraph(coord_list=data)

start_time = time.time()
for i in range(10):
    s = TSPSolver(g.dist_matrix, 8)
    s.execute(1000)
end_time = time.time()

print(1 / s.best, s.best_solution)
results['ch150.tsp'].append((round((end_time - start_time)/10, 5), round(1 / s.best, 3)))

52445.54299999998 [13, 62, 68, 97, 116, 49, 100, 128, 0, 41, 140, 47, 114, 26, 145, 130, 91, 10, 101, 84, 134, 38, 139, 5, 87, 3, 124, 86, 81, 40, 99, 125, 74, 48, 104, 111, 148, 146, 58, 118, 138, 29, 66, 18, 93, 57, 23, 8, 32, 107, 89, 95, 39, 85, 60, 108, 106, 65, 27, 90, 82, 11, 136, 73, 51, 33, 80, 115, 35, 102, 120, 133, 96, 7, 113, 112, 142, 6, 4, 147, 110, 126, 105, 42, 54, 61, 132, 83, 149, 123, 119, 144, 25, 21, 12, 94, 76, 15, 127, 131, 34, 64, 53, 28, 63, 109, 17, 121, 129, 122, 72, 75, 16, 46, 31, 36, 52, 77, 103, 92, 30, 9, 135, 98, 24, 20, 56, 141, 55, 78, 69, 43, 1, 37, 45, 44, 14, 143, 50, 71, 117, 137, 19, 2, 67, 22, 88, 70, 59, 79]


In [16]:
data = create_TSP_data('lab3_data', 4)
g = TSPGraph(coord_list=data)

start_time = time.time()
for i in range(10):
    s = TSPSolver(g.dist_matrix, 8)
    s.execute(1000)
end_time = time.time()

print(1 / s.best, s.best_solution)
results['fl417.tsp'].append((round((end_time - start_time)/10, 5), round(1 / s.best, 3)))

53559.11800000004 [128, 353, 344, 360, 322, 272, 208, 265, 60, 237, 305, 218, 82, 182, 232, 327, 300, 242, 378, 164, 33, 68, 349, 192, 351, 350, 400, 329, 66, 165, 358, 171, 271, 368, 336, 123, 20, 205, 132, 183, 343, 330, 202, 248, 101, 335, 273, 256, 352, 90, 212, 405, 411, 414, 185, 366, 224, 52, 47, 238, 298, 169, 347, 239, 96, 293, 119, 269, 397, 261, 31, 107, 279, 210, 278, 289, 380, 24, 220, 216, 337, 78, 72, 94, 354, 73, 281, 3, 391, 53, 241, 258, 67, 35, 186, 156, 118, 311, 74, 275, 105, 2, 126, 221, 304, 1, 385, 115, 249, 63, 203, 348, 284, 396, 184, 179, 372, 34, 266, 319, 102, 325, 92, 135, 374, 178, 341, 285, 113, 133, 58, 207, 91, 124, 407, 158, 95, 254, 170, 365, 117, 276, 112, 198, 136, 392, 125, 324, 231, 331, 404, 302, 277, 219, 174, 389, 383, 99, 71, 147, 176, 381, 88, 359, 22, 27, 373, 146, 388, 41, 306, 159, 253, 8, 297, 151, 106, 141, 234, 188, 69, 235, 80, 214, 0, 342, 29, 65, 225, 103, 131, 387, 255, 6, 70, 361, 142, 161, 318, 377, 215, 247, 175, 15, 321, 199, 4

In [17]:
table_time = [[' ', 'TSP genetic algorithm, 1000 populations']]
    
for benchmark in results:
    table_time.append([benchmark] + list(map(lambda x: f'{x[0]} / {x[1]}', results[benchmark])))

print('Measurments of running time of TSP genetic algorithm and results (time, s / answer)')
print(tabulate(table_time, tablefmt='fancy_grid'))

Measurments of running time of TSP genetic algorithm and results (time, s / answer)
╒════════════╤═════════════════════════════════════════╕
│            │ TSP genetic algorithm, 1000 populations │
├────────────┼─────────────────────────────────────────┤
│ a280.tsp   │ 1.64083 / 2808.732                      │
├────────────┼─────────────────────────────────────────┤
│ att48.tsp  │ 0.10386 / 153918.637                    │
├────────────┼─────────────────────────────────────────┤
│ bays29.tsp │ 0.0541 / 5620.0                         │
├────────────┼─────────────────────────────────────────┤
│ ch150.tsp  │ 0.59153 / 52445.543                     │
├────────────┼─────────────────────────────────────────┤
│ fl417.tsp  │ 3.53715 / 53559.118                     │
╘════════════╧═════════════════════════════════════════╛
